# TwitterTrends-4-MongoDBtoGMaps

## Importing libraries

In [15]:
from ipywidgets import interact, interactive, interact_manual, widgets, Label, HBox
from IPython.display import display, clear_output
import gmaps
import sys

import pymongo
from pymongo import MongoClient
from datetime import datetime

## Creating MongoDB connection, database and collection variables

In [18]:
client = MongoClient("mongodb://localhost/")
db = client["TwitterTrends"] # database
trends = db["trends"] # collection

### Set some indicators

In [19]:
total_count = trends.find().count() # total number of documents in trends collection

## Gmaps

### Loading Google Maps JavaScript API key
Because Google Maps APIs [no longer support keyless access](https://developers.google.com/maps/pricing-and-plans/standard-plan-2016-update) you need to provide your Google Maps JavaScript API key in order to call Google Maps API.

My API key is stored in *../files/secret.py* file:

In [48]:
sys.path.append('../files/')
import secret
gmaps.configure(api_key=secret.GMAPS_KEY) # Your Google API key

### Define functions

In [45]:
def init_Variables():
    global info_box_template, trend_info, locations, filterCountries
    
    info_box_template = """
<dl>
<dd><b>{country}</b>, {city}.</dd>
<dt>Trends at {date}</dt><dd>{trends}</dd>
</dl>
    """
    trend_info = []
    locations = []
    filterCountries = []  

# add to filterCountries array the countries wich are checked
def checkedCountries():
    if Spain_check.value == True: filterCountries.append("Spain")
    if UK_check.value == True: filterCountries.append("United Kingdom")
    if US_check.value == True: filterCountries.append("United States")
    if Japan_check.value == True: filterCountries.append("Japan")
    if Russia_check.value == True: filterCountries.append("Russia")
    if SA_check.value == True: filterCountries.append("South Africa")
    if Germany_check.value == True: filterCountries.append("Germany")
    if Argentina_check.value == True: filterCountries.append("Argentina")
    if Turkey_check.value == True: filterCountries.append("Turkey")
    if UAE_check.value == True: filterCountries.append("United Arab Emirates")

# main function called in jupyter's interact_manual method
def show_analysis(dateFrom, dateUntil, numLimit):
    global m, info_box_template, trend_info, locations, filterCountries, filtered_count
    
    m.close()  # closing possible previous gmap
    m = gmaps.Map()
    
    init_Variables()
    
    # Applying needed filters
    dateFromFilter = datetime(int(dateFrom.split("/")[2]), int(dateFrom.split("/")[1]), int(dateFrom.split("/")[0]), 0, 0, 0)
    dateUntilFilter = datetime(int(dateUntil.split("/")[2]), int(dateUntil.split("/")[1]), int(dateUntil.split("/")[0]), 0, 0, 0)
    
    checkedCountries()
    
    # Filter only by date
    if filterCountries == []:
        trends_filtered = trends.find(
                        {"$and": [ {"date": {"$gte": dateFromFilter}},
                                   {"date": {"$lte": dateUntilFilter}} ]
                        }).sort([("date", -1)]).limit(numLimit)
    # Filter by date and N countries checked by the user
    else:
        trends_filtered = trends.find(
                        {"$and": [ {"date": {"$gte": dateFromFilter}},
                                   {"date": {"$lte": dateUntilFilter}},
                                   {"$or": [ {"country": { "$in": filterCountries } } ] }
                                 ]}).sort([("date", -1)]).limit(numLimit)
        
    
    filtered_count = trends_filtered.count()
    
    print("Total number of documents in collection: " + str(total_count))
    print("Number of filtered documents: " + str(filtered_count))
    if numLimit < filtered_count:
        print("Limiting to: " + str(numLimit) + " number of documents")


    # Itering the MongoDB documents in order to feed gmaps info
    for trend in trends_filtered:
        trend_info += [info_box_template.format(
            country=trend["country"], city=trend["city"], date=trend["date"],
            #trends=trend["trends"])]
            trends=", ".join(trend["trends"].split(',')[:10]))]
         #   trends=trend["trends"].replace(',', ', '))]
        locations += [[trend["lat"], trend["lon"]]]

    m = gmaps.Map()
    marker_layer = gmaps.marker_layer(locations, info_box_content=trend_info)
    m.add_layer(marker_layer)
    display(m)

In [46]:
# creates the countries' checks
Spain_check = widgets.Checkbox(value=False, description='Spain', disabled=False)
UK_check = widgets.Checkbox(value=False, description='United Kingdom', disabled=False)
US_check = widgets.Checkbox(value=False, description='United States', disabled=False)
Japan_check = widgets.Checkbox(value=False, description='Japan', disabled=False)
Russia_check = widgets.Checkbox(value=False, description='Russia', disabled=False)
SA_check = widgets.Checkbox(value=False, description='South Africa', disabled=False)
Germany_check = widgets.Checkbox(value=False, description='Germany', disabled=False)
Argentina_check = widgets.Checkbox(value=False, description='Argentina', disabled=False)
Turkey_check = widgets.Checkbox(value=False, description='Turkey', disabled=False)
UAE_check = widgets.Checkbox(value=False, description='United Arab Emirates', disabled=False)

## Main program

Now we can filter by date or limit the number of results.

Optionally you can filter the results by any of the next countries:

In [51]:
# Show country's checks
display(HBox([Spain_check, UK_check, US_check, Japan_check, Russia_check]))
display(HBox([SA_check, Germany_check, Argentina_check, Turkey_check, UAE_check]))

# Create gmaps wigdet
m = gmaps.Map()

# Show interact button, in order to execute the date's filter and show gmaps widget
interact_manual(show_analysis, dateFrom="31/12/2000", dateUntil="31/12/2099", numLimit=(0, 1000, 10))

<function __main__.show_analysis>

--------------

### Examples

Here's a example of some trends in differents countries and languages:

![twitterTrends](../images/twitterTrends.png "twitterTrends")

Also, you can interact with this HTML example with some trends from different locations: 
- [twitterTrends.html](https://nbviewer.jupyter.org/github/rvilla87/Big-Data/blob/master/files/twitterTrends.html)

*Note: The html file was exported as [documented in jupyter-gmaps docs](http://jupyter-gmaps.readthedocs.io/en/latest/export.html#exporting-to-html).*